# Sesión 09: Análisis de regresión: variables dicotómicas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

## 9.1. Variables dicotómicas

El archivo *b08_ubermty* contiene información de viajes en taxi o uber en la ciudad de Monterrey de junio de 2016 a agosto de 2017. Crearemos un modelo para estimar la duración de cada viaje en función de la distancia y el día de la semana.

In [ ]:
# Cargar los datos
df = pd.read_excel('data/b08_ubermty.xlsx', usecols=['id', 'dropoff_longitude', 'dropoff_latitude', 
                                                     'Dia', 'DuracionMinutos', 'DistanciaKm'])

In [ ]:
# Revisar las variables
df.info()

In [ ]:
# Revisar las primeras filas
df.head()

In [ ]:
# Obtener la estadística descriptiva
df.describe().T

In [ ]:
# Filtrado de valores atípicos
Q1 = df['DuracionMinutos'].quantile(0.25)
Q3 = df['DuracionMinutos'].quantile(0.75)
IQR = Q3-Q1
lim_inf = Q1-1.5*IQR
lim_sup = Q3+1.5*IQR
df = df[(df['DuracionMinutos']>lim_inf)&(df['DuracionMinutos']<lim_sup)]

In [ ]:
# Generar un gráfico
sns.scatterplot(x="dropoff_longitude",
                y="dropoff_latitude",
                data=df)

In [ ]:
sns.boxplot(x='Dia', y='DuracionMinutos', data=df);

In [ ]:
df.head()

**Creación de variables dicotómicas**: La función `get_dummies` convierte una variable categórica (con k categorías) en k variables dicotómicas. Entre otros parámetros se puede establecer:
- prefix: agrega un prefijo a los nombres de las columnas.
- dtype: de manera predeterminada está en *booleano*, se puede cambiar a *int*
- drop_first: quita la columna con la primera categoría

In [ ]:
# Crear variables dicotómicas
dummy_vars = pd.get_dummies(df['Dia'], prefix='Dia', dtype=int, drop_first=True)
dummy_vars.head(3)

In [ ]:
# Agregar las variables dummy al DataFrame original
df = pd.concat([df, dummy_vars], axis=1)
# Quitar variable categórica original
df = df.drop(columns=['Dia'])

In [ ]:
df.columns

In [ ]:
X = df[['DistanciaKm', 'Dia_2', 'Dia_3', 'Dia_4', 'Dia_5',
       'Dia_6', 'Dia_7']]  
# Añadir una columna de unos para el intercepto
X = sm.add_constant(X)
y = df['DuracionMinutos']

In [ ]:
# Ajustar el modelo de regresión
model = sm.OLS(y, X).fit()

# Imprimir los resultados
print(model.summary())